In [1]:
def localParslConfig_threadpool(params):
 
    import parsl
    from parsl.executors import ThreadPoolExecutor
    from parsl.config import Config

    import os
    workingdir = params['working_dir']
    rundir = os.path.join(workingdir, 'runinfo')
    parsl.clear()
    local_tpex = Config(
        executors=[
            ThreadPoolExecutor(
                label="tpex_Local",
                max_threads=8,
                thread_name_prefix='tpex_run',
                working_dir=workingdir,
            )
        ],
        strategy=None,
        run_dir=rundir
    )

    return(local_tpex)

In [2]:
2+2

4